<a href="https://colab.research.google.com/github/RFajardoMonzon/MachineLearningCourse/blob/master/Sentiment_Analysis_An%C3%A1lisis_de_Sentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 15 - Análisis de sentimiento.


*   Recuerda que puedes consultar la documentación sobre una función escribiendo **?** justo después de la función: *Ejemplo: np.maximum?*
*   Puedes ejecutar el contenido de una celda con el atajo de teclado **CTRL+ENTER**
*   Utiliza **TAB** cada vez que quieras autocompletar una llamada a una función.
*   Puedes ejecutar instrucciones de bash directamente desde el notebook usando **!** : *Ejemplo: !pip install tensorflow*
*   Recuerda que Google es tu amigo, y saber buscar la información en las documentaciones de las librerías es muy importante.
*   Una solución correcta no es la que funciona sino la que se entiende!
*   No dudes en preguntar cualquier duda al profesor que lleva todo el día dando la turra.

## 1. ¿Te ha gustado esta película?


Hoy vamos a trabajar con reviews de películas recopilas de la web IMDB. El objetivo será el de analizar el texto que describe la review de un usuario, y al mismo tiempo ver qué puntuación este le ha dado a la película. Buscaremos entrenar a un modelo que sepa entender el sentimiento asociado al texto escrito por un usuario *¿Es una descripción positiva, negativa, neutra?*

> > > > > <img src=https://www.samyzaf.com/ML/imdb/review1.png width=500px>

1. Comencemos analizando un poco nuestro *dataset* viendo cuáles son las palabras que con mayor frecuencia se repiten. Para ello, elige 1000 reviews de manera aleatoria, y contabiliza la frecuencia de sus palabras. Después, de ese listado, imprime cuáles son las **50 palabras de mayor frecuencia**. ¿Tienen sentido?
2. Ahora ya podemos crear nuestro modelo para resolver este problema de clasificación. Has diferentes pruebas con diferentes tipos de arquitectura (i.e. con capas *LSTM()*, tipo multicapa, con *Embedders()*) y comprueba el rendimiento medido en accuracy y tiempo de entrenamiento de cada una de ellas. Busca una respuesta intuitiva de tus experimentos. Cuando tengas tu modelo entrenado, haz pruebas con texto tuyos propios.
3. **(Bonus) :** Ahora puedes probar a usar capas de Embeddings ya pre-entrenados. Utiliza Vec2Word o GloVe en tu arquitectura y comprueba qué mejoras te aporta.

In [0]:
import numpy as np
old = np.load

np.load = lambda *a,**k: old(*a, allow_pickle=True, **k)

In [0]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing
from tensorflow.keras.utils import to_categorical

max_features = 100 # Número máximo de palabras diferentes de nuestro vocabulario.
maxlen = 300         # Número máximo de palabras en cada texto.

# Carga el texto ya tokenizado.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features) 

# Preprocesado de los textos para igualar todas sus longitudes.
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test =  preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen) 

# Utiliza esta línea para obtener la palabra asociada a cada índice.
# imdb.get_word_index()

In [0]:
word_index = imdb.get_word_index()

In [0]:
keys = [key for key, _ in word_index.items()]

In [0]:
keys_dict = {v: k for k, v in word_index.items()}

In [0]:
print(len(keys))
print(len(keys_dict))

88584
88584


In [0]:
def get_max_indexes(array, size):
  res = []
  freqs = []
  for _ in np.arange(size):
    max_arg = np.argmax(array)
    freqs.append(array[max_arg])
    array[max_arg] = -1
    res.append(max_arg)
  return np.array(res), np.array(freqs)

In [0]:
random_reviews = x_train[np.random.choice(x_train.shape[0], 1000, replace=False)]
bincounts = np.array([np.bincount(review, minlength=len(keys_dict)+1) for review in random_reviews])

# print(bincounts.shape)

total_bincounts = np.sum(bincounts, axis=0)
max_indexes, freqs = get_max_indexes(total_bincounts[1:], 50)
print(max_indexes)
most_frequent = np.array([keys_dict[word+1] for word in max_indexes])
print([[w, v] for w, v in zip(most_frequent, freqs)])

[ 1  3  4  5  6  7  8  9 10 12 11 13 14 15 16 19 18 17 20 21 22 23 24 25
 27 29 26 28 30 31 32  0 33 34 35 37 36 38 39 40 45 41 42 44 47 49 43 50
 46 48]
[['and', 11009], ['of', 10985], ['to', 5366], ['is', 5216], ['br', 4637], ['in', 4345], ['it', 3531], ['i', 3490], ['this', 3106], ['was', 2678], ['that', 2610], ['as', 2531], ['for', 2192], ['with', 1602], ['movie', 1517], ['on', 1516], ['film', 1447], ['but', 1440], ['not', 1400], ['you', 1208], ['are', 1155], ['his', 1021], ['have', 1007], ['he', 994], ['one', 925], ['at', 900], ['be', 888], ['all', 872], ['by', 868], ['an', 819], ['they', 753], ['the', 742], ['who', 730], ['so', 726], ['from', 703], ['her', 700], ['like', 669], ['or', 661], ['just', 659], ['about', 607], ['some', 593], ["it's", 587], ['out', 580], ['if', 580], ['what', 575], ['more', 544], ['has', 536], ['when', 527], ['there', 520], ['good', 513]]


In [0]:
nModel = 4

In [0]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *

In [0]:


model = Sequential()

model.add(Embedding(max_features, 128))

model.add(CuDNNLSTM(128))

model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer=SGD(1.5), loss="binary_crossentropy", metrics=["acc"])

model.fit(x_train, y_train, epochs=8, validation_data=(x_test, y_test))

model.save("LSTM{}.h5".format(nModel))

nModel += 1

Train on 25000 samples, validate on 25000 samples
Epoch 1/8
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.6815 - acc: 0.5462 - val_loss: 0.7055 - val_acc: 0.5272
Epoch 2/8
25000/25000 [==============================] - 49s 2ms/sample - loss: 0.6107 - acc: 0.6692 - val_loss: 0.7166 - val_acc: 0.5669
Epoch 3/8
25000/25000 [==============================] - 49s 2ms/sample - loss: 0.4935 - acc: 0.7611 - val_loss: 0.3545 - val_acc: 0.8471
Epoch 4/8
25000/25000 [==============================] - 49s 2ms/sample - loss: 0.3386 - acc: 0.8546 - val_loss: 0.5060 - val_acc: 0.7877
Epoch 5/8
25000/25000 [==============================] - 49s 2ms/sample - loss: 0.2693 - acc: 0.8900 - val_loss: 0.3715 - val_acc: 0.8476
Epoch 6/8
25000/25000 [==============================] - 49s 2ms/sample - loss: 0.2071 - acc: 0.9186 - val_loss: 0.2869 - val_acc: 0.8864
Epoch 7/8
25000/25000 [==============================] - 50s 2ms/sample - loss: 0.1613 - acc: 0.9381 - val_loss: 0.2815 - 

In [0]:
model_fc = Sequential()

model_fc.add(Embedding(max_features, 32, input_shape=(maxlen,)))
model_fc.add(Flatten())

# model_fc.summary()

model_fc.add(Dense(4, activation="relu"))
model_fc.add(Dense(1, activation="sigmoid"))

model_fc.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

model_fc.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

model_fc.save("fc{}.h5".format(nModel))

nModel += 1

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 6s 249us/sample - loss: 0.6933 - acc: 0.4974 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/20
25000/25000 [==============================] - 6s 236us/sample - loss: 0.6932 - acc: 0.5018 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/20
25000/25000 [==============================] - 6s 236us/sample - loss: 0.6932 - acc: 0.4980 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/20
25000/25000 [==============================] - 6s 236us/sample - loss: 0.6933 - acc: 0.5006 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 5/20
25000/25000 [==============================] - 6s 244us/sample - loss: 0.6395 - acc: 0.6431 - val_loss: 0.5986 - val_acc: 0.7008
Epoch 6/20
25000/25000 [==============================] - 6s 246us/sample - loss: 0.5438 - acc: 0.7421 - val_loss: 0.5867 - val_acc: 0.6996
Epoch 7/20
25000/25000 [==============================] - 6s 237us/sample - loss: 0.4815 - acc: 0.7863 - val_l

In [0]:
from tensorflow.keras.backend import clear_session

clear_session()

model_oh = Sequential()

model_oh.add(Flatten())

model_oh.add(Dense(32, activation="relu"))
model_oh.add(Dense(8, activation="relu"))
model_oh.add(Dense(1, activation="sigmoid"))

model_oh.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

# print(x_train.shape, x_test.shape)

model_oh.fit(to_categorical(x_train), y_train, epochs=20, validation_data=(to_categorical(x_test), y_test))

model_oh.save("oh{}.h5".format(nModel))

nModel += 1

Train on 25000 samples, validate on 25000 samples
Epoch 1/20
25000/25000 [==============================] - 10s 392us/sample - loss: 0.6741 - acc: 0.5706 - val_loss: 0.6670 - val_acc: 0.5871
Epoch 2/20
25000/25000 [==============================] - 9s 380us/sample - loss: 0.4978 - acc: 0.7636 - val_loss: 0.6482 - val_acc: 0.6514
Epoch 3/20
25000/25000 [==============================] - 10s 390us/sample - loss: 0.3066 - acc: 0.8698 - val_loss: 0.7889 - val_acc: 0.6456
Epoch 4/20
25000/25000 [==============================] - 10s 387us/sample - loss: 0.2040 - acc: 0.9178 - val_loss: 1.0261 - val_acc: 0.6422
Epoch 5/20
25000/25000 [==============================] - 9s 380us/sample - loss: 0.1308 - acc: 0.9504 - val_loss: 1.2673 - val_acc: 0.6342
Epoch 6/20
25000/25000 [==============================] - 9s 379us/sample - loss: 0.0825 - acc: 0.9711 - val_loss: 1.5719 - val_acc: 0.6379
Epoch 7/20
25000/25000 [==============================] - 9s 377us/sample - loss: 0.0515 - acc: 0.9828 - va

In [0]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 852.3MB 46.9MB/s 
  Stored in directory: /tmp/pip-ephem-wheel-cache-hwmet2gs/wheels/0d/bc/67/e6a9108ab86cd076703af19ad4e0f02f57381ac6583df16249
Successfully built en-core-web-lg

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_lg -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-05-30 12:49:43--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.164.117
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.164.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  58.4MB/s    in 23s     

2019-05-30 12:50:06 (69.4 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
!gunzip 'GoogleNews-vectors-negative300.bin.gz'

In [0]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)



In [0]:
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,max_features)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=max_features:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)


In [0]:
print(embedding_matrix.shape)

(100, 300)


In [0]:
from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

Using TensorFlow backend.
